In [391]:
from datetime import datetime
import pandas as pd
import yaml
from sqlalchemy import create_engine

## Database Connection

In [392]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [393]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [394]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction

In [395]:
cliente = pd.read_sql_table('dim_cliente', etl_conn)
mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
sede = pd.read_sql_table('dim_sede', etl_conn)  
tiempo = pd.read_sql_table('dim_tiempo', etl_conn)
servicio = pd.read_sql_table("mensajeria_servicio",url_db)

In [396]:
total_nan = servicio.isna().sum()
print(total_nan)

id                                    0
descripcion                           0
nombre_solicitante                    0
fecha_solicitud                       0
hora_solicitud                        0
fecha_deseada                         6
hora_deseada                          0
nombre_recibe                         0
telefono_recibe                       0
descripcion_pago                    260
ida_y_regreso                         0
activo                                0
novedades                           336
cliente_id                            0
destino_id                            0
mensajero_id                        727
origen_id                             0
tipo_pago_id                          0
tipo_servicio_id                      0
tipo_vehiculo_id                      0
usuario_id                            0
prioridad                             0
ciudad_destino_id                     0
ciudad_origen_id                      0
hora_visto_por_mensajero          28428


In [397]:
print(len(servicio))

28430


In [398]:
servicio.head(5)

,id,descripcion,nombre_solicitante,fecha_solicitud,hora_solicitud,fecha_deseada,hora_deseada,nombre_recibe,telefono_recibe,descripcion_pago,...,ciudad_origen_id,hora_visto_por_mensajero,visto_por_mensajero,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado
0,34,recoger muestras a las 10 am,chat_GPT,2023-10-26,09:46:03,2023-10-26,09:46:03,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,
1,35,Recogervmx a las,chat_GPT,2023-10-26,11:18:14,2023-10-26,11:18:14,Gemini,100,,...,1,None,,,NaN,NaN,False,False,True,
2,36,recoger mx a las 9 am,chat_GPT,2023-10-28,19:21:01,2023-10-30,19:21:01,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,
3,41,recoger mx a las 11 am,chat_GPT,2023-11-07,09:46:09,2023-11-07,09:46:09,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,
4,42,recoger mx a las 11 am,chat_GPT,2023-11-07,09:46:10,2023-11-07,09:46:10,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,


## Transformation

In [399]:
tiempo_unique = tiempo[["month_year", "key_tiempo"]].drop_duplicates(subset=["month_year"])
servicio["año_mes"] = servicio["fecha_solicitud"].dt.to_period("M").astype(str)
servicio = servicio[["id", "año_mes", "cliente_id", "mensajero_id"]]
servicio.fillna(-1, inplace=True)
# Hacer el merge asegurando que coincidan
hecho_servicio_mensual = pd.merge(servicio, tiempo_unique[["month_year", "key_tiempo"]], left_on="año_mes", right_on="month_year", how="left")
hecho_servicio_mensual.head(5)
#Si es granularidad mensual, como debe aparecer la fecha en la tabla de hechos?
#Como lo tengo está mal porque la fehca está por día
#Las dimensiones deben pueden tener llaves foráneas, sede tiene llave foránea del cliente, las otras llaves se las quité
#Las dimensiones se deben mezclar?
#La dimensión tiempo está bien hecha?

,id,año_mes,cliente_id,mensajero_id,month_year,key_tiempo
0,34,2023-10,5,-1.0,2023-10,6360
1,35,2023-10,5,7.0,2023-10,6360
2,36,2023-10,5,-1.0,2023-10,6360
3,41,2023-11,5,-1.0,2023-11,7104
4,42,2023-11,5,-1.0,2023-11,7104


In [400]:
hecho_servicio_mensual = pd.merge(hecho_servicio_mensual, cliente[["cliente_id", "key_cliente"]], left_on="cliente_id", right_on="cliente_id", how="left")
hecho_servicio_mensual.head(5)

,id,año_mes,cliente_id,mensajero_id,month_year,key_tiempo,key_cliente
0,34,2023-10,5,-1.0,2023-10,6360,7
1,35,2023-10,5,7.0,2023-10,6360,7
2,36,2023-10,5,-1.0,2023-10,6360,7
3,41,2023-11,5,-1.0,2023-11,7104,7
4,42,2023-11,5,-1.0,2023-11,7104,7


In [401]:
mensajero.rename(columns={'id':'mensajero_id'}, inplace=True)
hecho_servicio_mensual = hecho_servicio_mensual.merge(mensajero[["mensajero_id", "key_mensajero"]], left_on="mensajero_id", right_on="mensajero_id", how="left")
hecho_servicio_mensual.fillna(-1, inplace=True)
hecho_servicio_mensual.head(5)

,id,año_mes,cliente_id,mensajero_id,month_year,key_tiempo,key_cliente,key_mensajero
0,34,2023-10,5,-1.0,2023-10,6360,7,-1.0
1,35,2023-10,5,7.0,2023-10,6360,7,13.0
2,36,2023-10,5,-1.0,2023-10,6360,7,-1.0
3,41,2023-11,5,-1.0,2023-11,7104,7,-1.0
4,42,2023-11,5,-1.0,2023-11,7104,7,-1.0


In [402]:
print(len(hecho_servicio_mensual))

28430


In [403]:
sede_unique = sede[["cliente_id", "key_sede"]].drop_duplicates(subset=["cliente_id"])
hecho_servicio_mensual = pd.merge(hecho_servicio_mensual, sede_unique[["cliente_id", "key_sede"]], left_on="cliente_id", right_on="cliente_id", how="left")
hecho_servicio_mensual.head(5)

,id,año_mes,cliente_id,mensajero_id,month_year,key_tiempo,key_cliente,key_mensajero,key_sede
0,34,2023-10,5,-1.0,2023-10,6360,7,-1.0,16
1,35,2023-10,5,7.0,2023-10,6360,7,13.0,16
2,36,2023-10,5,-1.0,2023-10,6360,7,-1.0,16
3,41,2023-11,5,-1.0,2023-11,7104,7,-1.0,16
4,42,2023-11,5,-1.0,2023-11,7104,7,-1.0,16


In [404]:
print(len(hecho_servicio_mensual))

28430


## Cálculo del atributo solicitudes_por_mes

In [405]:
# Contar solicitudes por cada 'año_mes'
solicitudes_por_mes = hecho_servicio_mensual.groupby("año_mes").size().reset_index(name="solicitud_por_mes")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original
hecho_servicio_mensual = hecho_servicio_mensual.merge(solicitudes_por_mes, on="año_mes", how="left")
hecho_servicio_mensual.head(5)

,id,año_mes,cliente_id,mensajero_id,month_year,key_tiempo,key_cliente,key_mensajero,key_sede,solicitud_por_mes
0,34,2023-10,5,-1.0,2023-10,6360,7,-1.0,16,12
1,35,2023-10,5,7.0,2023-10,6360,7,13.0,16,12
2,36,2023-10,5,-1.0,2023-10,6360,7,-1.0,16,12
3,41,2023-11,5,-1.0,2023-11,7104,7,-1.0,16,17
4,42,2023-11,5,-1.0,2023-11,7104,7,-1.0,16,17


In [406]:
print(len(hecho_servicio_mensual))

28430


## Cálculo del atributo solicitudes_por_mes_cliente

In [407]:
# Contar solicitudes por cada 'key_cliente'
solicitud_por_mes_cliente = hecho_servicio_mensual.groupby("key_cliente").size().reset_index(name="solicitud_por_mes_cliente")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original
hecho_servicio_mensual = hecho_servicio_mensual.merge(solicitud_por_mes_cliente, on="key_cliente", how="left")
hecho_servicio_mensual.head(5)

,id,año_mes,cliente_id,mensajero_id,month_year,key_tiempo,key_cliente,key_mensajero,key_sede,solicitud_por_mes,solicitud_por_mes_cliente
0,34,2023-10,5,-1.0,2023-10,6360,7,-1.0,16,12,4651
1,35,2023-10,5,7.0,2023-10,6360,7,13.0,16,12,4651
2,36,2023-10,5,-1.0,2023-10,6360,7,-1.0,16,12,4651
3,41,2023-11,5,-1.0,2023-11,7104,7,-1.0,16,17,4651
4,42,2023-11,5,-1.0,2023-11,7104,7,-1.0,16,17,4651


In [408]:
print(len(hecho_servicio_mensual))

28430


## Cálculo del atributo solicitudes_por_mes_sede

In [409]:
# Contar solicitudes por cada 'key_sede'
solicitudes_por_mes_sede = hecho_servicio_mensual.groupby("key_sede").size().reset_index(name="solicitud_por_mes_sede")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original
hecho_servicio_mensual = hecho_servicio_mensual.merge(solicitudes_por_mes_sede, on="key_sede", how="left")
hecho_servicio_mensual.head(5)

,id,año_mes,cliente_id,mensajero_id,month_year,key_tiempo,key_cliente,key_mensajero,key_sede,solicitud_por_mes,solicitud_por_mes_cliente,solicitud_por_mes_sede
0,34,2023-10,5,-1.0,2023-10,6360,7,-1.0,16,12,4651,4651
1,35,2023-10,5,7.0,2023-10,6360,7,13.0,16,12,4651,4651
2,36,2023-10,5,-1.0,2023-10,6360,7,-1.0,16,12,4651,4651
3,41,2023-11,5,-1.0,2023-11,7104,7,-1.0,16,17,4651,4651
4,42,2023-11,5,-1.0,2023-11,7104,7,-1.0,16,17,4651,4651


In [410]:
print(len(hecho_servicio_mensual))

28430


## Cálculo del atributo solicitudes_por_mes_mensajero

In [411]:
# Contar solicitudes por cada 'key_mensajero'
solicitudes_por_mes_mensajero = hecho_servicio_mensual.groupby("key_mensajero").size().reset_index(name="solicitud_por_mes_mensajero")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original
hecho_servicio_mensual = hecho_servicio_mensual.merge(solicitudes_por_mes_mensajero, on="key_mensajero", how="left")
hecho_servicio_mensual.head(10)

,id,año_mes,cliente_id,mensajero_id,month_year,key_tiempo,key_cliente,key_mensajero,key_sede,solicitud_por_mes,solicitud_por_mes_cliente,solicitud_por_mes_sede,solicitud_por_mes_mensajero
0,34,2023-10,5,-1.0,2023-10,6360,7,-1.0,16,12,4651,4651,727
1,35,2023-10,5,7.0,2023-10,6360,7,13.0,16,12,4651,4651,68
2,36,2023-10,5,-1.0,2023-10,6360,7,-1.0,16,12,4651,4651,727
3,41,2023-11,5,-1.0,2023-11,7104,7,-1.0,16,17,4651,4651,727
4,42,2023-11,5,-1.0,2023-11,7104,7,-1.0,16,17,4651,4651,727
5,43,2023-11,5,-1.0,2023-11,7104,7,-1.0,16,17,4651,4651,727
6,46,2023-11,4,12.0,2023-11,7104,6,24.0,0,17,117,117,436
7,45,2023-11,5,12.0,2023-11,7104,7,24.0,16,17,4651,4651,436
8,47,2023-11,5,12.0,2023-11,7104,7,24.0,16,17,4651,4651,436
9,44,2023-11,5,-1.0,2023-11,7104,7,-1.0,16,17,4651,4651,727


In [412]:
hecho_servicio_mensual.drop(columns=["cliente_id","año_mes","mensajero_id","month_year"], inplace=True)
hecho_servicio_mensual

,id,key_tiempo,key_cliente,key_mensajero,key_sede,solicitud_por_mes,solicitud_por_mes_cliente,solicitud_por_mes_sede,solicitud_por_mes_mensajero
0,34,6360,7,-1.0,16,12,4651,4651,727
1,35,6360,7,13.0,16,12,4651,4651,68
2,36,6360,7,-1.0,16,12,4651,4651,727
3,41,7104,7,-1.0,16,17,4651,4651,727
4,42,7104,7,-1.0,16,17,4651,4651,727
...,...,...,...,...,...,...,...,...,...
28425,28267,13680,11,8.0,4,4304,17384,17384,1059
28426,28363,13680,11,27.0,4,4304,17384,17384,1252
28427,28403,13680,7,32.0,16,4304,4651,4651,2439
28428,28438,13680,11,18.0,4,4304,17384,17384,73


In [413]:
hecho_servicio_mensual.to_sql("hecho_servicio_mensual", etl_conn, if_exists="replace", index_label="key_solicitud_servicio") 

430